In [ ]:
import os
import cv2
import numpy as np

def create_dataset(video_dir, label, frame_count=10, frame_size=(256, 256)):
    videos = []
    labels = []
    i=0
    for video_file in os.listdir(video_dir):
        video_path = os.path.join(video_dir, video_file)
        cap = cv2.VideoCapture(video_path)
        frames = []
        i+=1
        print(i, end=" ")
        if i == 1000:
            break
        # Read the specified number of frames
        while len(frames) < frame_count:
            ret, frame = cap.read()
            if not ret:
                break
            # Resize frame
            frame = cv2.resize(frame, frame_size)
            frames.append(frame)
        
        cap.release()
        
        # Only add the video if it has the required number of frames
        if len(frames) == frame_count:
            videos.append(np.array(frames))  # Convert frames to NumPy array
            labels.append(label)
    
    return videos, labels  # Return as lists, not as NumPy arrays

# Example usage:
real_videos_dir = '/kaggle/input/celeb-df-v2/Celeb-real'
fake_videos_dir = '/kaggle/input/celeb-df-v2/Celeb-synthesis'

# Create datasets
real_videos, real_labels = create_dataset(real_videos_dir, label=1)
fake_videos, fake_labels = create_dataset(fake_videos_dir, label=0)

# Combine real and fake datasets
videos = real_videos + fake_videos
labels = real_labels + fake_labels

# Convert lists to NumPy arrays after combining
videos = np.array(videos)
labels = np.array(labels)

print(f"Shape of video dataset: {videos.shape}")
print(f"Shape of labels: {labels.shape}")


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv3D, BatchNormalization, MaxPooling3D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model

class DeepFakeDetectionModel:
    def __init__(self, input_shape):
        self.input_shape = input_shape
        self.model = self.build_model()

    def build_model(self):
        inputs = Input(shape=self.input_shape)

        # Convolutional Layer 1
        x = Conv3D(8, (3, 3, 3), activation='relu', padding='same', name='conv3d_1')(inputs)
        x = BatchNormalization(name='batch_normalization_1')(x)
        x = MaxPooling3D((2, 2, 2), name='max_pooling3d_1')(x)

        # Convolutional Layer 2
        x = Conv3D(16, (3, 3, 3), activation='relu', padding='same', name='conv3d_2')(x)
        x = BatchNormalization(name='batch_normalization_2')(x)
        x = MaxPooling3D((2, 2, 2), name='max_pooling3d_2')(x)

        # Convolutional Layer 3
        x = Conv3D(32, (3, 3, 3), activation='relu', padding='same', name='conv3d_3')(x)
        x = BatchNormalization(name='batch_normalization_3')(x)
        x = MaxPooling3D((2, 2, 2), name='max_pooling3d_3')(x)

        # Flatten
        x = Flatten(name='flatten')(x)

        # Dense Layer
        x = Dense(64, activation='relu', name='dense_1')(x)
        x = Dropout(0.5, name='dropout')(x)

        # Output Layer
        outputs = Dense(1, activation='sigmoid', name='output')(x)

        # Create Model
        model = Model(inputs=inputs, outputs=outputs, name='DeepFakeDetectionModel')

        # Compile Model
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        return model

    def summary(self):
        return self.model.summary()

    def train(self, train_data, validation_data, epochs=150, batch_size=16):
        history = self.model.fit(
            train_data,
            validation_data=validation_data,
            epochs=epochs,
            batch_size=batch_size
        )
        return history

    def evaluate(self, test_data):
        return self.model.evaluate(test_data)

    def predict(self, data):
        return self.model.predict(data)

    def save(self, filepath):
        self.model.save(filepath)

    def load(self, filepath):
        self.model = tf.keras.models.load_model(filepath)

# Example usage:
# Define input shape according to your data
frame_count = 10
height = width = 256
channels = 3
input_shape = (frame_count, height, width, channels)  # (10, 256, 256, 3)

# Initialize the model
deepfake_detector = DeepFakeDetectionModel(input_shape)

# Print model summary
deepfake_detector.summary()

# Prepare your data
# Create tf.data.Dataset or use numpy arrays directly
train_data = tf.data.Dataset.from_tensor_slices((videos, labels)).batch(16)
validation_data = tf.data.Dataset.from_tensor_slices((videos, labels)).batch(16)  # Replace with actual validation data

# Train the model
deepfake_detector.train(train_data, validation_data, epochs=28)

# Evaluate the model (example)
# deepfake_detector.evaluate(test_data)



# Save the model (example)
deepfake_detector.save('deepfake_detection_model.h5')

# Load the model (example)
deepfake_detector.load('deepfake_detection_model.h5')


In [ ]:
import os

file_path = '/kaggle/input/celeb-df-v2/Celeb-synthesis/id17_id30_0004.mp4'

# Check if the file exists
if os.path.exists(file_path):
    print("File exists.")
else:
    print("File does not exist.")


In [ ]:
import cv2
import numpy as np

# Function to preprocess the frames
def preprocess_frame(frame):
    img = cv2.resize(frame, (256, 256))
    img = img.astype('float32') / 255.0
    return img

# Function to detect deepfake in a video
def detect_deepfake(video_path, frame_count=10):
    cap = cv2.VideoCapture(video_path)
    frames = []
    deepfake_scores = []
    i=0
    while True  :
        i += 1
        if i==21 :
            break
        ret, frame = cap.read()
        if not ret:
            break
        
        # Preprocess the frame
        img = preprocess_frame(frame)
        
        # Accumulate frames until we have enough
        frames.append(img)
        
        # If we have enough frames, predict
        if len(frames) == frame_count:
            # Convert frames list to a NumPy array with shape (frame_count, height, width, channels)
            frames_array = np.array(frames)
            
            # Predict using the model
            prediction = deepfake_detector.predict(np.expand_dims(frames_array, axis=0))  # Add batch dimension
            deepfake_scores.append(prediction[0][0])
            # Remove the first frame from the list to process the next sequence
            frames.pop(0)

    cap.release()
    
    # Calculate the average score
    avg_score = np.mean(deepfake_scores) if deepfake_scores else 0
    print(f"Average Deepfake Score: {avg_score}")

    # Determine if the video is deepfake or real
    if avg_score > 0.5:
        print("The video is likely a Deepfake.")
    else:
        print("The video is likely Real.")

# Example usage
video_path = '/kaggle/input/celeb-df-v2/YouTube-real/00000.mp4'
# Ensure the model is properly loaded before calling detect_deepfake
# For example:
# model = DeepFakeDetectionModel(input_shape=(10, 256, 256, 3)).model
# model.load_weights('path_to_weights.h5')  # or model.load('path_to_model.h5')

# Detect deepfake
detect_deepfake(video_path)
